In [1]:
from urllib.parse import urlparse
from urllib.parse import parse_qs
import math
from youtube_transcript_api import YouTubeTranscriptApi
from summarizer import Summarizer
import torch
from transformers import AutoTokenizer, AutoModelWithLMHead
import numpy as np
import pandas as pd
import os
import torch
import json 
from transformers import T5Tokenizer, T5ForConditionalGeneration, T5Config
from summarizer import Summarizer,TransformerSummarizer
from transformers import BartForConditionalGeneration, BartTokenizer
import textwrap

In [2]:
yt_video_list=[]

In [3]:
import scrapetube

#dummy channel id = UCcQNe3F_pVgxL45jsAfCxxg
# videos = scrapetube.get_channel("UCVMJPRXrBIOFDBZEaF-_pEA")
videos = scrapetube.get_channel("UCVMJPRXrBIOFDBZEaF-_pEA")

# for video in videos:
#     print(video['videoId'])  #directly get the video id 
# print(type(videos))
list_of_video_id=[]
for video in videos:
    list_of_video_id.append(video['videoId'])  #directly get the video id 
    # print(video.keys())
    # print(video['videoId'])
    # url='https://www.youtube.com/watch?v='+video['videoId']

In [4]:
print(videos)

<generator object get_channel at 0x2b13a5700>


In [5]:
yt_video_list=list_of_video_id

In [6]:
print(len(yt_video_list))

178


In [7]:
list_of_chunks_all_videos=[]    
for video_id in list_of_video_id:
    list_of_lines=[]  #for current video
    # url_data=urlparse(video_url)
    # query = parse_qs(url_data.query)
    # video_id = query["v"][0]
    # print(video_id)
    video_url='https://www.youtube.com/watch?v='+video['videoId']
    try:
        transcritps = YouTubeTranscriptApi.get_transcript(video_id,languages=['en'])
        for i in transcritps:
            time=i['start']
            time=math.floor(time)
            hours=int(time//3600)
            minutes=int((time%3600)//60)
            secs=time%60
            # print( f"{hours:02}:{minutes:02}:{secs:06.3f}")
            # i['start']=f"{hours:02}:{minutes:02}:{secs:06.3f}" #streamlit doesnt need this
            i['start']=time
            ## and adding youtube video
            i['video_url']=video_url   # adding youtube video url
            list_of_lines.append({'time':i['start'],'text':i['text'],'video_url':i['video_url']})
        list_of_chunks=[]
        chunk=[]
        chunk.append(list_of_lines[0])
        chunk.append(list_of_lines[1])
        no=0
        for i in list_of_lines[2:]:
            chunk.append(i)
            no+=1
            if len(chunk)>8:
                chunk.pop(0)
            if (no)%6==0:
                chunk_start=chunk[0]['time']
                chunk_url=chunk[0]['video_url']
                chunk_text=''
                count=0
                for j in chunk:
                    count+=1
                    chunk_text+=j['text']
                    chunk_text+=' '
                list_of_chunks.append({'time':chunk_start,'text':chunk_text,'video_url':video_url})

                no=0
        if len(chunk)!=0:
            chunk_start=chunk[0]['time']
            chunk_url=chunk[0]['video_url']
            chunk_text=''
            count=0
            for j in chunk:
                count+=1
                chunk_text+=j['text']
                chunk_text+=' '
            # print(count)
            # print(chunk_text)
            # print()
            list_of_chunks.append({'time':chunk_start,'text':chunk_text,'video_url':video_url})

        no=0
        # print(len(list_of_chunks))
        list_of_chunks_all_videos.extend(list_of_chunks)
        print("the len now is ",len(list_of_chunks_all_videos))
    except:
        print("didnt work for : ", video_id)
        continue
    

the len now is  78
the len now is  91
the len now is  103
the len now is  117
the len now is  224
the len now is  240
the len now is  341
the len now is  365
the len now is  464
the len now is  479
the len now is  597
the len now is  620
didnt work for :  zFCd3QemC6o
the len now is  627
the len now is  645
the len now is  664
the len now is  681
the len now is  703
the len now is  717
the len now is  732
the len now is  832
the len now is  847
the len now is  951
the len now is  968
the len now is  989
the len now is  1088
the len now is  1106
the len now is  1128
the len now is  1140
the len now is  1152
the len now is  1269
the len now is  1354
the len now is  1597
the len now is  1746
the len now is  1863
the len now is  1970
the len now is  1987
the len now is  2005
the len now is  2015
the len now is  2106
the len now is  2128
the len now is  2146
the len now is  2165
the len now is  2184
the len now is  2202
the len now is  2283
the len now is  2308
the len now is  2423
the len n

In [20]:
len(list_of_chunks_all_videos)

8802

In [21]:
import pandas as pd

In [22]:
df=pd.DataFrame(list_of_chunks_all_videos)

In [23]:
df.shape

(8802, 3)

In [24]:
print(df.head(20)['text'][0])
print(df.head(20)['text'][1])

so I was thinking we could come up with some fun branding for this like fake Music Festival I went ahead and kind of came up with this really rough pencil sketch the first thing I want to do is get this rounded text I'm going to use good old text and a path tool Coastline so I'm just going to line 
good old text and a path tool Coastline so I'm just going to line these points to make sure I can get everything centered to kind of create the 3D look I'm going to create outlines for this and then I'm going to go to the Pathfinder tool and merge all this now I'm going to go to object path offset 


In [25]:
df.head()

,time,text,video_url
0,0,so I was thinking we could come up with some f...,https://www.youtube.com/watch?v=54tWSvfa8j0
1,14,good old text and a path tool Coastline so I'm...,https://www.youtube.com/watch?v=54tWSvfa8j0
2,28,Pathfinder tool and merge all this now I'm goi...,https://www.youtube.com/watch?v=54tWSvfa8j0
3,44,change this to White and then I'm going to opt...,https://www.youtube.com/watch?v=54tWSvfa8j0
4,57,specified steps so let's just try a there you ...,https://www.youtube.com/watch?v=54tWSvfa8j0


In [26]:
import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)
warnings.filterwarnings("ignore", message="Unverified HTTPS request")
warnings.filterwarnings("ignore", message="InsecureRequestWarning")

In [27]:
from opensearchpy import OpenSearch

CLUSTER_URL = 'https://localhost:9200'

def get_os_client(cluster_url = CLUSTER_URL,
                  username='admin',
                  password='admin'):
    '''
    Get OpenSearch client
    :param cluster_url: cluster URL like https://ml-te-netwo-1s12ba42br23v-ff1736fa7db98ff2.elb.us-west-2.amazonaws.com:443
    :return: OpenSearch client
    '''
    client = OpenSearch(
        hosts=[cluster_url],
        http_auth=(username, password),
        verify_certs=False
    )
    return client

client = get_os_client()

/Users/vpatwal/Desktop/trying_summarization/.venv/lib/python3.10/site-packages/opensearchpy/connection/http_urllib3.py:208: UserWarning: Connecting to https://localhost:9200 using SSL with verify_certs=False is insecure.
  warnings.warn(


In [28]:
from sentence_transformers import SentenceTransformer

MODEL_NAME = "sentence-transformers/msmarco-distilbert-base-tas-b"


In [29]:
model = SentenceTransformer(MODEL_NAME)

In [30]:
EMBEDDING_DIM = model.encode(["Sample sentence"])[0].shape[0]
print(EMBEDDING_DIM)#will be used ahead

768


In [31]:
index_name = "all_videos"

index_body = {
  "settings": {
    "index": {
      "knn": True,
      "knn.algo_param.ef_search": 100
    }
  },
  "mappings": { #how do we store, 
    "properties": {
        "embedding": {
          "type": "knn_vector", #we are going to put 
          "dimension": EMBEDDING_DIM,
          # "dimension":800,
          "method": {
            "name": "hnsw",
            "space_type": "l2",
            "engine": "nmslib",
            "parameters": {
              "ef_construction": 128,
              "m": 24
            }
         }
     }
    }
}}


In [33]:
response = client.indices.delete(index = index_name)

In [34]:
response = client.indices.create(index=index_name, body=index_body)

In [35]:
import pandas as pd

for index, row in df.iterrows():
    print(f"Id: {row['time']}, Start: {row['time']}, Overview: {row['text']},url:{row['video_url']}")
    # original_title = row['time']
    overview = row['text']
    id = row['time']
    st_time=row['time']
    url=row['video_url']
    # Sentence transformer model takes list of documents as input and returns list of embeddings.
    embedding = model.encode([overview])[0]
    # We are inserting a data point with 3 attribute, "id", "text" and "embedding" as knn_vector type.
    my_doc = {"id": id, "plot": overview, "embedding": embedding,"time" : st_time,"url":url}
    res = client.index(
        index=index_name,
        body=my_doc,
        id = str(index),
        refresh = True
        )

Id: 0, Start: 0, Overview: so I was thinking we could come up with some fun branding for this like fake Music Festival I went ahead and kind of came up with this really rough pencil sketch the first thing I want to do is get this rounded text I'm going to use good old text and a path tool Coastline so I'm just going to line ,url:https://www.youtube.com/watch?v=54tWSvfa8j0
Id: 14, Start: 14, Overview: good old text and a path tool Coastline so I'm just going to line these points to make sure I can get everything centered to kind of create the 3D look I'm going to create outlines for this and then I'm going to go to the Pathfinder tool and merge all this now I'm going to go to object path offset ,url:https://www.youtube.com/watch?v=54tWSvfa8j0
Id: 28, Start: 28, Overview: Pathfinder tool and merge all this now I'm going to go to object path offset path yeah there you go let's do you got .15 I'm going to merge this to and just change the color I'm going to go ahead and add that outline st

In [39]:
""" Example query text """
# user_query = "Applying Lens Blur"
# user_query = "hide content"
# user_query = "expanding"
# user_query="deleting people"
# user_query='How does the Generative AI Remove feature in Lightroom work?'
# user_query="What steps does Jacob describe for creating a drop shadow in Illustrator?"
# user_query="Inserting drop shadow"
# user_query="cut it"
# user_query="How to create a montage using markers in Premiere Pro?"
# user_query="select and arrange photos for a video montage?"
# user_query="add markers to a timeline for precise editing"
# user_query="delete unwanted sections of a video sequence in Premiere Pro"
# user_query="Asset Availability and Customization"
# user_query="saving the version"
user_query="image tracing elements"


""" Embedding the query by using the same model """
query_embedding = model.encode((user_query))


query_body = {
    "query": {"knn": {"embedding": {"vector": query_embedding, "k": 10}}},
    "_source": False,
    "fields": ["id", "title", "plot","time","url"],
    "size":10  # vivek added this , top 5
}

results = client.search(
    body=query_body,
    index=index_name
)

In [40]:
for i, result in enumerate(results["hits"]["hits"]):
    # print(i,results)
    plot = result['fields']['plot'][0]
    url = result['fields']['url'][0]
    score = result['_score']
    time=result['fields']['time'][0]
    print(f"{i+1}., Start Time: {time} Score: {score}, Url: {url}  Plot: {plot}")

1., Start Time: 202 Score: 0.022823513, Url: https://www.youtube.com/watch?v=54tWSvfa8j0  Plot: really quickly and just like matching all these shapes that we have and these not need to be perfect because I feel like the imperfections kind of add to the illustration almost because if I use image Trace which is basically tracing an image I think it would be really cool to see the result but I'm going to press 
2., Start Time: 847 Score: 0.01956801, Url: https://www.youtube.com/watch?v=54tWSvfa8j0  Plot: then maybe we want to use image Trace so where's image Trace image Trace sits here at the top click on image trace and it says this is a large image I can size it down maybe let's do that actually let's keep it large because I want to compare it to the shapes that I made click okay and it's already image 
3., Start Time: 937 Score: 0.019438386, Url: https://www.youtube.com/watch?v=54tWSvfa8j0  Plot: regardless so I'm just quickly implementing that effect that we talked about by just rost

In [41]:
df.to_csv('all_videos_table.csv')